## Ingeniería de características

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
from sklearn.preprocessing import LabelEncoder
import joblib

In [2]:
df = pd.read_csv("./dataset/violenciaIntrafamiliarGuatemala2008_2023_v2.csv")

FileNotFoundError: [Errno 2] No such file or directory: './dataset/violenciaIntrafamiliarGuatemala2008_2023_v2.csv'

## Dropear columnas

In [ ]:
df = df.drop([
    'VIC_OCUP_INE', 'AGR_OCUP_INE', 'VIC_OCUP_GRUPO',
       'VIC_OCUP_SUB_GRUPO_PRINCIPAL', 'VIC_OCUP_SUB_GRUPO',
       'VIC_OCUP_GRUPO_UNITARIO', 'AGR_OCUP_GRUPO',
       'AGR_OCUP_SUB_GRUPO_PRINCIPAL', 'AGR_OCUP_SUB_GRUPO',
       'AGR_OCUP_GRUPO_UNITARIO',
       'VIC_DEDICA', 'AGR_DEDICA', 'AGR_OCUP', 'VIC_OCUP'
    
    ],
        axis=1)

### Disminuir la variable de HIJOS a rangos

In [ ]:
def rangos_hijos(num):
    if num == 0:
        return "sin hijos"
    elif num == 1:
        return "hijo unico"
    elif 1<num <4:
        return "hijos medios"
    elif 3<num:
        return "muchos hijos"
    elif num==9999.0:
        return "ignorado"
df["CANTIDAD_HIJOS"] = df["TOTAL_HIJOS"].apply(lambda x: rangos_hijos(x))
def con_tipo_hijos(x):
    if x == 0:
        return "si"
    elif x == 9999.0:
        return "ignorado"
    else:
        return "no"
df["CON_HIJ_HOM"] = df["NUM_HIJ_HOM"].apply(lambda x: con_tipo_hijos(x))
df["CON_HIJ_MUJ"] = df["NUM_HIJ_MUJ"].apply(lambda x: con_tipo_hijos(x))

### Disminuir la variable de EDAD a rangos

In [ ]:
def rango_edad(num):
    if num <= 13:
        return "ninio"
    elif num <= 18:
        return "adolescente"
    elif num <= 40:
        return "adulto joven"
    elif num <= 60:
        return "adulto medio"
    elif num == 9999.0:
        return "ignorado"
    else:
        return "adulto mayor"
df["VIC_RANGO_EDAD"] = df["VIC_EDAD"].apply(lambda x: rango_edad(x))
df["AGR_RANGO_EDAD"] = df["AGR_EDAD"].apply(lambda x: rango_edad(x))

### Disminuir la variable de ESCOLARIDAD a nivel

In [ ]:
def nivel_escolaridad(nivel):
    if re.search("basico", nivel):
        return "basicos"
    elif re.search("primaria", nivel):
        return "primaria"
    elif re.search("universitario", nivel):
        return "nivel_medio_o_superior"
    elif re.search("(diversificado|divesificado)", nivel):
        return "nivel_medio_o_superior"
    return nivel
df["VIC_NIV_ESCOLARIDAD"] = df["VIC_ESCOLARIDAD"].apply(lambda x: nivel_escolaridad(x))
df["AGR_NIV_ESCOLARIDAD"] = df["AGR_ESCOLARIDAD"].apply(lambda x: nivel_escolaridad(x))
df["VIC_ESCOLARIDAD"] = df["VIC_ESCOLARIDAD"].apply(lambda x: nivel_escolaridad(x))
df["AGR_ESCOLARIDAD"] = df["AGR_ESCOLARIDAD"].apply(lambda x: nivel_escolaridad(x))

### Disminuir la variable NUMERO_DE_OTROS_AGRESORES a rangos

In [ ]:
def rango_otros_agresores(num):
    if num == 0:
        return "no"
    elif num == 99:
        return "ignorado"
    else:
        return "si"

df["OTROS_AGRESORES"] = df["AGRESORES_OTROS_TOTAL"].apply(lambda x: rango_otros_agresores(x))
df["OTROS_AGRESORES_N_AS"] = df["AGR_OTRAS_N_AS"].apply(lambda x: rango_otros_agresores(x))
df["OTROS_AGRESORES_N_OS"] = df["AGR_OTROS_N_OS"].apply(lambda x: rango_otros_agresores(x))
df["OTROS_AGRESORES_MUJ"] = df["AGR_OTRAS_MUJ"].apply(lambda x: rango_otros_agresores(x))
df["OTROS_AGRESORES_HOM"] = df["AGR_OTROS_HOM"].apply(lambda x: rango_otros_agresores(x))

### Disminuir la variable de OTRAS_VICTIMAS a rangos

In [ ]:
def rango_otras_victimas(num):
    if num == 0:
        return "no"
    elif num == 9999.0:
        return "ignorado"
    else:
        return "si"

df["VIC_OTRAS_VICTIMAS"] = df["OTRAS_VICTIMAS"].apply(lambda x: rango_otras_victimas(x))
df["VIC_OTRAS_VICTIMAS_HOM"] = df["VIC_OTRAS_HOM"].apply(lambda x: rango_otras_victimas(x))
df["VIC_OTRAS_VICTIMAS_MUJ"] = df["VIC_OTRAS_MUJ"].apply(lambda x: rango_otras_victimas(x))
df["VIC_OTRAS_VICTIMAS_N_OS"] = df["VIC_OTRAS_N_OS"].apply(lambda x: rango_otras_victimas(x))
df["VIC_OTRAS_VICTIMAS_N_AS"] = df["VIC_OTRAS_N_AS"].apply(lambda x: rango_otras_victimas(x))

### Construir una variable para determinar los casos de violencia donde el tipo sea físico, psicológico, patrimonial o sexual

In [ ]:
# Crear nuevas columnas booleanas
df["VIOLENCIA_FISICA"] = df["HEC_TIPAGRE"].str.contains("fisica").astype(str).replace({"True":"presente", "False":"no presente"})
df["VIOLENCIA_PSICOLOGICA"] = df["HEC_TIPAGRE"].str.contains("psicologica").astype(str).replace({"True":"presente", "False":"no presente"})
df["VIOLENCIA_SEXUAL"] = df["HEC_TIPAGRE"].str.contains("sexual").astype(str).replace({"True":"presente", "False":"no presente"})
df["VIOLENCIA_PATRIMONIAL"] = df["HEC_TIPAGRE"].str.contains("patrimonial").astype(str).replace({"True":"presente", "False":"no presente"})

### Construir una variable para establecer la diferencia de edad entre la víctima y el agresor

In [ ]:
def get_age_difference(row):
    if row["AGR_EDAD"] >=9999 or row["VIC_EDAD"] >=9999:
        return 9999
    return abs(row["AGR_EDAD"] - row["VIC_EDAD"])
    
def get_who_is_bigger(row):
    if row["AGR_EDAD"] >=9999 or row["VIC_EDAD"] >=9999:
        return "ignorado"
    elif row["AGR_EDAD"] > row["VIC_EDAD"]:
        return "agresor"
    elif row["AGR_EDAD"] < row["VIC_EDAD"]:
        return "victima"
    else:
        return "iguales"
    
df["DIF_EDAD_VIC_AGR"] = df.apply(get_age_difference, axis=1)
df["QUIEN_ES_MAYOR"] = df.apply(get_who_is_bigger, axis=1)

### Construir una variable que muestre la diferencia de alfabetizacion

In [ ]:
def get_difference_alfab(row):
    if row["AGR_ALFAB"] == "ignorado" or row["VIC_ALFAB"] == "ignorado":
        return "ignorado"
    elif row["AGR_ALFAB"] == row["VIC_ALFAB"]:
        return "iguales"
    elif row["AGR_ALFAB"] == "alfabeta":
        return "agresor"
    elif row["VIC_ALFAB"] == "alfabeta":
        return "victima"

df["DIF_ALFAB"] = df.apply(get_difference_alfab, axis=1)

### Construir una variable que muestre la diferencia de la fecha del registro de la agresion contra la fecha cuando sucedio la agresion

In [ ]:
# Mapeo de meses en español a números
MESES_MAP = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12
}

def get_difference_dates_registro_hec(row):
    hec_ano = row["HEC_ANO"]
    hec_mes = row["HEC_MES"]
    hec_dia = row["HEC_DIA"]

    if hec_ano == 9999:
        return "ignorado"

    try:
        mes_emision = MESES_MAP.get(row["MES_EMISION"].lower())
        fecha_emision = datetime(int(row["ANO_EMISION"]), mes_emision, int(row["DIA_EMISION"]))
    except Exception as e:
        return "fecha emisión inválida"

    # Si el mes es ignorado
    if hec_mes == "ignorado":
        if hec_ano != row["ANO_EMISION"]:
            diff_years = row["ANO_EMISION"] - hec_ano
            if diff_years < 0:
                return "fecha futura"
            elif diff_years == 0:
                return "ignorado"
            elif diff_years == 1:
                return "1 año"
            elif diff_years <= 5:
                return "5 años"
            elif diff_years <= 10:
                return "10 años"
            else:
                return "más de 10 años"
        else:
            return "ignorado"

    mes_hecho = MESES_MAP.get(hec_mes.lower())
    if mes_hecho is None:
        return "mes inválido"

    # Si el día es desconocido, aproximar usando meses
    if hec_dia == 99:
        diff_months = (fecha_emision.year - hec_ano) * 12 + (fecha_emision.month - mes_hecho)
        diff_days = diff_months * 30
    else:
        try:
            fecha_hecho = datetime(int(hec_ano), mes_hecho, int(hec_dia))
            diff_days = (fecha_emision - fecha_hecho).days
        except Exception:
            return "fecha hecho inválida"

    # Clasificación según diferencia en días
    if diff_days < 0:
        return "fecha futura"
    elif diff_days <= 7:
        return "una semana"
    elif diff_days <= 30:
        return "un mes"
    elif diff_days <= 90:
        return "3 meses"
    elif diff_days <= 180:
        return "6 meses"
    elif diff_days <= 365:
        return "1 año"
    elif diff_days <= 5 * 365:
        return "5 años"
    elif diff_days <= 10 * 365:
        return "10 años"
    else:
        return "más de 10 años"

# Aplicar la función
df["DIF_TIEMPO_REGISTRO_HECHO"] = df.apply(get_difference_dates_registro_hec, axis=1)

### Construir una variable que mida la diferencia de sexo

In [ ]:
def get_dif_sex(row):
    if row["VIC_SEXO"] == row["AGR_SEXO"]:
        return "no"
    return "si"
df["DIF_SEXO"] = df.apply(get_dif_sex, axis=1)

### Construir variable que mida la diferencia de etnia

In [ ]:
def get_dif_sex(row):
    if row["VIC_GRUPET"] == row["AGR_GRUPET"]:
        return "si"
    return "no"
df["DIF_GRUPET"] = df.apply(get_dif_sex, axis=1)

In [ ]:
def es_indigena(val):
    if val == "ladinos(as)": return "no"
    elif val == "ignorado" or val == "no indica": return "ignorado"
    else: return "si"
df["VIC_ES_INDIGENA"] = df["VIC_GRUPET"].apply(lambda val: es_indigena(val))
df["AGR_ES_INDIGENA"] = df["AGR_GRUPET"].apply(lambda val: es_indigena(val))

### Construir variable que mida la diferencia de escolaridad
El negativo significa que la victima tiene una mayor escolaridad, mientras que positivo significa que el agresor tiene un mayor grado de escolaridad

In [ ]:
def diferencia_escolaridad(row):
    niveles = {
        "ninguno": 0,
        "primaria": 1,
        "basicos": 2,
        "diversificado": 3,
        "universidad": 4
    }

    agr = row["AGR_NIV_ESCOLARIDAD"]
    vic = row["VIC_NIV_ESCOLARIDAD"]

    if agr == "ignorado" or vic == "ignorado":
        return "ignorado"

    nivel_agr = niveles.get(agr.lower())
    nivel_vic = niveles.get(vic.lower())

    if nivel_agr is None or nivel_vic is None:
        return "nivel inválido"

    return nivel_agr - nivel_vic

# Aplicar la función al DataFrame
df["DIF_ESCOLARIDAD_AGR_VIC"] = df.apply(diferencia_escolaridad, axis=1)


In [ ]:
escolaridad_orden = {
    "ninguno": 0,
    "primero primaria": 1,
    "segundo primaria": 2,
    "tercero primaria": 3,
    "cuarto primaria": 4,
    "quinto primaria": 5,
    "sexto primaria": 6,
    "primaria grado ignorado": 1,  # suponemos inicio

    "primero basico": 7,
    "segundo basico": 8,
    "tercero basico": 9,
    "basico grado ignorado": 7,  # suponemos inicio

    "cuarto diversificado": 10,
    "quinto diversificado": 11,
    "sexto diversificado": 12,
    "diversificado grado ignorado": 10,  # suponemos inicio

    "primer ano universitario": 13,
    "segundo ano universitario": 14,
    "tercer ano universitario": 15,
    "cuarto ano universitario": 16,
    "quinto ano universitario": 17,
    "sexto ano universitario": 18,
    "septimo ano universitario": 19,
    "universitario grado ignorado": 13,  # suponemos inicio

    "ignorado": None
}
def diferencia_escolaridad_detallada(row):
    escolaridad_orden = {
        "ninguno": 0,
        "primero primaria": 1,
        "segundo primaria": 2,
        "tercero primaria": 3,
        "cuarto primaria": 4,
        "quinto primaria": 5,
        "sexto primaria": 6,
        "primaria grado ignorado": 6,

        "primero basico": 7,
        "segundo basico": 8,
        "tercero basico": 9,
        "basico grado ignorado": 9,

        "cuarto diversificado": 10,
        "quinto diversificado": 11,
        "sexto diversificado": 12,
        "diversificado grado ignorado": 12,

        "primer ano universitario": 13,
        "segundo ano universitario": 14,
        "tercer ano universitario": 15,
        "cuarto ano universitario": 16,
        "quinto ano universitario": 17,
        "sexto ano universitario": 18,
        "septimo ano universitario": 19,
        "universitario grado ignorado": 19,

        "ignorado": None
    }

    agr = row["AGR_ESCOLARIDAD"]
    vic = row["VIC_ESCOLARIDAD"]

    nivel_agr = escolaridad_orden.get(agr.lower())
    nivel_vic = escolaridad_orden.get(vic.lower())

    if nivel_agr is None or nivel_vic is None:
        return "ignorado"

    return nivel_agr - nivel_vic

# Aplicar la función
df["DIF_ESCOLARIDAD_DETALLADA"] = df.apply(diferencia_escolaridad_detallada, axis=1)


### Identificar si gana más o menos la víctima que el agresor

In [ ]:
def gana_mas_la_victima(row:pd.Series):
    if row['VIC_SALARIO']>row['AGR_SALARIO']:
        return 'si'
    return 'no'
df['GANA_MAS_LA_VICTIMA'] = df.apply(gana_mas_la_victima, axis=1)

## Crear los label encodings para kmodes y LCA

In [ ]:
category_types = df.select_dtypes(exclude=["number", "float64", "int64", np.number]).columns

In [ ]:
df[category_types] = df[category_types].astype(str)

In [ ]:
encoders = {}
for col in category_types:
    le = LabelEncoder()
    df[f'{col}_ENC'] = le.fit_transform(df[col])
    encoders[col] = le
joblib.dump(encoders, './dataset/label_encoders.pkl')

C:\Users\diego\AppData\Local\Temp\ipykernel_17104\838065732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_ENC'] = le.fit_transform(df[col])
C:\Users\diego\AppData\Local\Temp\ipykernel_17104\838065732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_ENC'] = le.fit_transform(df[col])
C:\Users\diego\AppData\Local\Temp\ipykernel_17104\838065732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

['./dataset/label_encoders.pkl']

In [ ]:
df.to_csv('./dataset/violenciaIntrafamiliarGuatemala2008_2023_v3.csv', index=False)